In [ ]:
import wave
import struct
import numpy as np
import pyaudio
import librosa
import soundfile as sf

x, _ = librosa.load('4.wav', sr=16000)  # 输入文件1：音频1(调节音量）sr：采样率（可以改）

volume = 0.5  # 调节x的音量（倍数）
x = x * volume
sf.write('t1.wav', x, 16000)

y, _ = librosa.load('5.wav', sr=16000)  # 输入文件2：音频2
sf.write('t2.wav', y, 16000)
f1 = wave.open('t1.wav', 'rb')
f2 = wave.open('t2.wav', 'rb')

# 音频1的数据
params1 = f1.getparams()
nchannels1, sampwidth1, framerate1, nframes1, comptype1, compname1 = params1[:6]
print(nchannels1, sampwidth1, framerate1, nframes1, comptype1, compname1)
f1_str_data = f1.readframes(nframes1)
f1.close()
f1_wave_data = np.frombuffer(f1_str_data, dtype=np.int16)

# 音频2的数据
params2 = f2.getparams()
nchannels2, sampwidth2, framerate2, nframes2, comptype2, compname2 = params2[:6]
print(nchannels2, sampwidth2, framerate2, nframes2, comptype2, compname2)
f2_str_data = f2.readframes(nframes2)
f2.close()
f2_wave_data = np.frombuffer(f2_str_data, dtype=np.int16)

# 对不同长度的音频用数据零对齐补位
if nframes1 < nframes2:
    length = abs(nframes2 - nframes1)
    temp_array = np.zeros(length, dtype=np.int16)
    rf1_wave_data = np.concatenate((f1_wave_data, temp_array))
    rf2_wave_data = f2_wave_data
elif nframes1 > nframes2:
    length = abs(nframes2 - nframes1)
    temp_array = np.zeros(length, dtype=np.int16)
    rf2_wave_data = np.concatenate((f2_wave_data, temp_array))
    rf1_wave_data = f1_wave_data
else:
    rf1_wave_data = f1_wave_data
    rf2_wave_data = f2_wave_data

# 合并1和2的数据
new_wave_data = rf1_wave_data + rf2_wave_data
new_wave_data = new_wave_data * 1.0 / (max(abs(new_wave_data)))  # wave幅值归一化
new_wave = new_wave_data.tostring()

p = pyaudio.PyAudio()
CHANNELS = 1
FORMAT = pyaudio.paInt16

# 写文件
framerate = 44100
time = 10

# 打开WAV文档
f = wave.open(r".\6.wav", "wb")  # 文件3：输出音频

# 配置声道数、量化位数和取样频率
nchannels = 1  # 通道数 目前设置为1
sampwidth = 2
data_size = len(new_wave_data)
framerate = 16000  # 需要和前面的采样率保持一致
nframes = data_size
comptype = "NONE"
compname = "not compressed"
f.setparams((nchannels, sampwidth, framerate, nframes, comptype, compname))
# 将wav_data转换为二进制数据写入文件
# f.writeframes(new_wave)
for v in new_wave_data:
    f.writeframes(struct.pack('h', int(v * 64000 / 2)))
f.close()


